## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Tables

### Links: 

[[installation_point_st_partner]]

[[entries_installation_points_dir_partner]]

[[installation_point_st_partner]]

[[buildings_st_partner]]

[[gates_st_partner]]
___

In [5]:
query_text = """--sql
CREATE TABLE db1.t_all_installetion_points
(
    `address_uuid` String,
    `city` String,
    `country` String,
    `created_at` String,
    `full_address` String,
    `installation_point_id` Int32,
    `parent_uuid` String ,
    `region` String,
    `report_date` Date
)
    ENGINE = MergeTree()
    ORDER BY installation_point_id
"""

ch.query_run(query_text)

DatabaseError: HTTPDriver for https://rc1a-eflrt968scbm232q.mdb.yandexcloud.net:8443 received ClickHouse error code 57
 Code: 57. DB::Exception: Table db1.t_all_installetion_points already exists. (TABLE_ALREADY_EXISTS) (version 25.3.3.42 (official build))


In [6]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_all_installetion_points_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR 50 MINUTE TO db1.t_all_installetion_points AS
    SELECT
        parent_uuid,
        installation_point_id,
        address_uuid,
        report_date,
        full_address,
        created_at,
        region,
        country,
        city
    FROM (
        SELECT
            parent_uuid,
            installation_point_id,
            address_uuid,
            report_date,
            full_address,
            created_at,
            region,
            country,
            city
        FROM db1.installation_point_st_partner_ch as installation_point_st_partner
        LEFT JOIN db1.entries_installation_points_dir_partner_ch USING(installation_point_id)
        WHERE installation_point_st_partner.installation_point_id != 0 AND installation_point_st_partner.installation_point_id IS NOT NULL
        UNION ALL
        SELECT
            parent_uuid,
            installation_point_id,
            address_uuid,
            report_date,
            arrayStringConcat(arraySlice(splitByString(',', full_address), 1, length(splitByString(',', full_address)) - 1), ',') AS full_address,
            created_at,
            region,
            country,
            city
        FROM db1.buildings_st_partner_ch as buildings_st_partner
        LEFT JOIN db1.entries_installation_points_dir_partner USING(parent_uuid)
        WHERE buildings_st_partner.installation_point_id != 0 AND buildings_st_partner.installation_point_id IS NOT NULL
        UNION ALL
        SELECT
            parent_uuid,
            installation_point_id,
            address_uuid,
            report_date,
            arrayStringConcat(arraySlice(splitByString(',', full_address), 1, length(splitByString(',', full_address)) - 1), ',') AS full_address,
            created_at,
            region,
            country,
            city
        FROM db1.gates_st_partner as gates_st_partner
        LEFT JOIN db1.entries_installation_points_dir_partner USING(parent_uuid)
        WHERE gates_st_partner.installation_point_id != 0 AND gates_st_partner.installation_point_id IS NOT NULL
    )
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [7]:
query_text = """--sql
    SELECT
        *
    FROM db1.t_all_installetion_points
    ORDER BY report_date desc
    limit 100
    """

ch.query_run(query_text)


address_uuid,city,country,created_at,full_address,installation_point_id,parent_uuid,region,report_date
str,str,str,str,str,i64,str,str,date
"""0a2f9718-be59-4265-a635-1f431e…","""Иннополис""","""Россия""","""2023-01-25 15:15:12""","""Россия, Республика Татарстан, …",10,"""d13b04d0-0e88-4326-884c-7a511e…","""Республика Татарстан""",2025-08-19
"""029d2d1d-792e-4b43-8245-6c4693…","""Иннополис""","""Россия""","""2018-05-25 07:50:25""","""Россия, Республика Татарстан, …",9,"""172aff05-9d66-4f98-b485-1f177d…","""Республика Татарстан""",2025-08-19
"""82ae8710-c673-4ddd-b8b0-d88eb3…","""Иннополис""","""Россия""","""2018-05-25 07:50:25""","""Россия, Республика Татарстан, …",8,"""172aff05-9d66-4f98-b485-1f177d…","""Республика Татарстан""",2025-08-19
"""501b7f66-eb3d-410f-96d0-da75fc…","""Иннополис""","""Россия""","""2018-05-25 07:50:25""","""Россия, Республика Татарстан, …",7,"""172aff05-9d66-4f98-b485-1f177d…","""Республика Татарстан""",2025-08-19
"""a6b2cde0-26b5-4ce3-af2e-8d03e3…","""Иннополис""","""Россия""","""2018-05-25 07:50:25""","""Россия, Республика Татарстан, …",2,"""54086eea-e2b4-46ad-ba06-913233…","""Республика Татарстан""",2025-08-19
…,…,…,…,…,…,…,…,…
"""4b0b5d9d-9a3e-4771-b297-f7ffe4…","""Луховицы""","""Россия""","""2019-03-14 10:57:59""","""Россия, Московская область, го…",393,"""a2c69717-900f-4973-b47d-fb2446…","""Московская область""",2025-08-19
"""812e6d6f-e573-46de-b7f5-0481c3…","""Волгоград""","""Россия""","""2019-03-13 10:48:09""","""Россия, Волгоградская область,…",387,"""d18298ac-9e2d-4eba-a477-be16b7…","""Волгоградская область""",2025-08-19
"""525b5d32-707d-4e27-b000-9cbc15…","""Тюмень""","""Россия""","""2019-03-14 05:48:35""","""Россия, Тюменская область, гор…",392,"""4b6c527a-54ee-48ce-ae2b-d65a32…","""Тюменская область""",2025-08-19


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.all_installetion_points_parquet_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [5]:
query_text = """--sql
    DROP TABLE db1.all_installetion_points_parquet_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.all_installetion_points_parquet_ch
    """

ch.query_run(query_text)

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.all_installetion_points_parquet_mv
"""

ch.query_run(query_text)